In [1]:
import os

In [2]:
%pwd

'c:\\Users\\Admin\\Documents\\AI project\\AI_DL-project\\research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Admin\\Documents\\AI project\\AI_DL-project'

In [5]:
# mlflow >> to track experiments

In [6]:
#https://dagshub.com/Surajjj0030/AI_DL-project.mlflow



In [7]:
os.environ["MLFLOW_TRACKING_URI"]="https://dagshub.com/Surajjj0030/AI_DL-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"]="Surajjj0030"
os.environ["MLFLOW_TRACKING_PASSWORD"]="939c88a98ccc6f1d43ab2bcb1fb1fc1e8d854e56"

In [8]:
! pip install mlflow

  Using cached alembic-1.15.2-py3-none-any.whl.metadata (7.3 kB)
  Using cached sqlalchemy-2.0.40-cp312-cp312-win_amd64.whl.metadata (9.9 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached sqlparse-0.5.3-py3-none-any.whl.metadata (3.9 kB)
  Using cached greenlet-3.1.1-cp312-cp312-win_amd64.whl.metadata (3.9 kB)
  Using cached gitdb-4.0.12-py3-none-any.whl.metadata (1.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached smmap-5.0.2-py3-none-any.whl.metadata (4.3 kB)
  Using cached anyio-4.9.0-py3-none-any.whl.metadata (4.7 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
   ---------------------------------------- 0.0/28.2 MB ? eta -:--:--
   -- ------------------------------------- 1.6/28.2 MB 8.4 MB/s eta 0:00:04
   ----- ---------------------------------- 3.7/2

In [9]:
import mlflow
import os

In [10]:
# Set MLFLOW tracking uri
mlflow.set_tracking_uri("https://dagshub.com/Surajjj0030/AI_DL-project.mlflow")

# Set your dagshub crenditals (mlflow uses HTTP basic Auth)
os.environ["MLFLOW_TRACKING_USERNAME"]="Surajjj0030"
os.environ["MLFLOW_TRACKING_PASSWORD"]="939c88a98ccc6f1d43ab2bcb1fb1fc1e8d854e56"

In [11]:
import tensorflow as tf 

In [12]:
model = tf.keras.models.load_model("artifacts/training/model.h5")

In [13]:
# entity

In [14]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class EvaluationConfig:
    path_of_model: Path
    training_data: Path
    all_params: dict
    mlflow_uri: str
    params_image_size: list
    params_batch_size: int

In [15]:
# configuration manager

In [26]:
from src.cnnClassifier.constants import *
from src.cnnClassifier.utils.common import read_yaml,create_directories,save_json

In [27]:
class ConfigurationManager:
    def __init__(
        self, 
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        create_directories([self.config.artifacts_root])

    
    def get_evaluation_config(self) -> EvaluationConfig:
        eval_config = EvaluationConfig(
            path_of_model="artifacts/training/model.h5",
            training_data="artifacts/data_ingestion/kidney-ct-scan-image",
            mlflow_uri="https://dagshub.com/Surajjj0030/AI_DL-project.mlflow",
            all_params=self.params,
            params_image_size=self.params.IMAGE_SIZE,
            params_batch_size=self.params.BATCH_SIZE
        )
        return eval_config

In [28]:
# components


In [29]:
import tensorflow as tf
import mlflow
from pathlib import Path
import mlflow.keras
from urllib.parse import urlparse
import json

In [30]:
class Evaluation:
    def __init__(self, config: EvaluationConfig):
        self.config = config

    
    def _valid_generator(self):

        datagenerator_kwargs = dict(
            rescale = 1./255,
            validation_split=0.30
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation="bilinear"
        )

        valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator = valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset="validation",
            shuffle=False,
            **dataflow_kwargs
        )


    @staticmethod
    def load_model(path: Path) -> tf.keras.Model:
        return tf.keras.models.load_model(path)
    

    def evaluation(self):
        self.model = self.load_model(self.config.path_of_model)
        self._valid_generator()
        self.score = model.evaluate(self.valid_generator)
        self.save_score()

    def save_score(self):
        scores = {"loss": self.score[0], "accuracy": self.score[1]}
        save_json(path=Path("scores.json"), data=scores)

    
    def log_into_mlflow(self):
        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme
        
        with mlflow.start_run():
            mlflow.log_params(self.config.all_params)
            mlflow.log_metrics(
                {"loss": self.score[0], "accuracy": self.score[1]}
            )
            # Model registry does not work with file store
            if tracking_url_type_store != "file":

                # Register the model
                # There are other ways to use the Model Registry, which depends on the use case,
                # please refer to the doc for more information:
                # https://mlflow.org/docs/latest/model-registry.html#api-workflow
                mlflow.keras.log_model(self.model, "model", registered_model_name="VGG16Model")
            else:
                mlflow.keras.log_model(self.model, "model")

In [31]:
try:
    config = ConfigurationManager()
    eval_config = config.get_evaluation_config()
    evaluation = Evaluation(eval_config)
    evaluation.evaluation()
    evaluation.log_into_mlflow()

except Exception as e:
   raise e

Found 139 images belonging to 2 classes.


C:\Users\Admin\AppData\Roaming\Python\Python312\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


9/9 ━━━━━━━━━━━━━━━━━━━━ 34s 4s/step - accuracy: 0.3896 - loss: 1.5901


2025/04/14 14:52:43 WARNING mlflow.keras.save: You are saving a Keras model without specifying model signature.
2025/04/14 14:53:00 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.
Successfully registered model 'VGG16Model'.
2025/04/14 14:53:41 INFO mlflow.store.model_registry.abstract_store: Waiting up to 300 seconds for model version to finish creation. Model name: VGG16Model, version 1
Created version '1' of model 'VGG16Model'.


🏃 View run righteous-doe-243 at: https://dagshub.com/Surajjj0030/AI_DL-project.mlflow/#/experiments/0/runs/69f943743c6042279f89b7699973dd78
🧪 View experiment at: https://dagshub.com/Surajjj0030/AI_DL-project.mlflow/#/experiments/0
